In [1]:
from langchain.embeddings import OpenAIEmbeddings
from numpy import dot
from numpy.linalg import norm

In [3]:
#エンベッディングのモデル指定
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

#クエリーをベクトル化（１）
query_vecter = embeddings.embed_query("飛行車の最高速度は？")

#ベクトル変換された値を表示
print(f"ベクトル化された質問:{query_vecter[:5]}")

#違う文字列をベクトル化（２）
document_1_vector = embeddings.embed_query("飛行車の最高速度は時速150キロメートルです。")
#違う文字列をベクトル化（３）
document_2_vector = embeddings.embed_query("鶏肉和適切に下味をつけた後、中火で焼きながらたまに裏返し、 \
外側は香ばしく柔らかく仕上げる。")

#（１）と（２）の比較
cos_sim_1 = dot(query_vecter, document_1_vector)/(norm(query_vecter)*norm(document_1_vector))
print(f"ドキュメント１と質問の類似度:{cos_sim_1}")
#（２）と（３）の比較
cos_sim_2 = dot(query_vecter, document_2_vector)/(norm(query_vecter)*norm(document_2_vector))
print(f"ドキュメント２と質問の類似度:{cos_sim_2}")

ベクトル化された質問:[-0.0006812680055983601, 0.00824013646957589, -0.004933948253756882, -0.021167207898880762, -0.008620157996529503]
ドキュメント１と質問の類似度:0.9275586408305281
ドキュメント２と質問の類似度:0.7388375742485174


# 与えられたPDFを元に回答

In [5]:
!pip install pymupdf

In [4]:
from langchain.document_loaders import PyMuPDFLoader

In [5]:
#PDFファイルを読み込む
loader = PyMuPDFLoader("sample.pdf")
documents = loader.load()
#ドキュメント数表示
print(f"ドキュメントの数:{len(documents)}")
#一つ目のドキュメントの内容表示
print(f"1つ目のドキュメントの内容:{documents[0].page_content}")
#一つ目のドキュメントのメタデータ表示
print(f"1つ目のドキュメントのメタデータ:{documents[0].metadata}")

ドキュメントの数:12
1つ目のドキュメントの内容:飛行車に関する法制度
【注意】この文章は、架空の飛行車を対象にした法律の自動生成例です。

1つ目のドキュメントのメタデータ:{'source': 'sample.pdf', 'file_path': 'sample.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.4', 'title': '飛行車に関する法制度', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'macOS バージョン13.5（ビルド22G74） Quartz PDFContext, AppendMode 1.1', 'creationDate': '', 'modDate': "D:20230819044808Z00'00'", 'trapped': ''}


In [ ]:
#テキスト分割パッケージ
!pip install spacy

In [ ]:
#分割の対象となる言語（日本語）のモデル
!python -m spacy download ja_core_news_sm

In [14]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import SpacyTextSplitter

In [7]:
#
loader = PyMuPDFLoader("./sample.pdf")
documents = loader.load()
#
text_splitter=SpacyTextSplitter(
    chunk_size=300,
    pipeline="ja_core_news_sm"
)
#
splitted_documents = text_splitter.split_documents(documents)
#
print(f"分割前のドキュメント数:{len(documents)}")
print(f"分割後のドキュメント数:{len(splitted_documents)}")

分割前のドキュメント数:12
分割後のドキュメント数:56


In [15]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma

In [9]:
#
loader = PyMuPDFLoader("./sample.pdf")
documents = loader.load()
#
text_splitter=SpacyTextSplitter(
    chunk_size=300,
    pipeline="ja_core_news_sm"
)
#
splitted_documents = text_splitter.split_documents(documents)
#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)
#
database.add_documents(
    splitted_documents,
)
#
print("データベースの作成が完了しました！")

データベースの作成が完了しました！


In [10]:
#
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)
#
documents = database.similarity_search("飛行車の最高速度は？")
#
print(f"ドキュメント数:{len(documents)}")
#
for document in documents:
    #
    print(f"ドキュメントの内容:{document.page_content}")

ドキュメント数:4
ドキュメントの内容:飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容:飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容:飛行車速度制限法
第1条（目的）
本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する
基準を定めることを目的とする。




第2条（定義）
本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。




第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート
ルとする。


3.
特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ
ることがある。
ドキュメントの内容:第3条（一般的な速度制限）
1.
都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。


2.
都市部以外の地域において飛行車が飛行する場

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)

#
query = "飛行車の最高速度は？"
#
documents = database.similarity_search(query)
#
documents_string = ""
#
for document in documents:
    documents_string += f"""
    ------------------------
    {document.page_content}
    """
#
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

#
result = chat([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])
#
print(result.content)

都市部での飛行車の最高速度は時速150キロメートル、都市部以外の地域での飛行車の最高速度は時速250キロメートルとなります。


In [5]:
#from langchain.chat_models import ChatOpenAI
#from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma
#warning出たので変更
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)

#
query = "飛行車の最高速度は？"
#
documents = database.similarity_search(query)
#
documents_string = ""
#
for document in documents:
    documents_string += f"""
    ------------------------
    {document.page_content}
    """
#
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)

#
result = chat([
    HumanMessage(content=prompt.format(document=documents_string, query=query))
])
#
print(result.content)

都市部においては時速150キロメートル、都市部以外の地域では時速250キロメートルとなります。


## チャット画面を作成する

In [ ]:
#chainlitのインストール
!pip install chainlit

### 手順
1.↓以下のセルをコピーしpyファイルを作る

2.chainlit run [Pythonファイル名]

In [ ]:
#理由は不明chainlitのターミナルでインストールするように言われる
!pip install langchain-community

In [ ]:
#chainlitの簡易テスト
import chainlit as cl

@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="準備ができました！メッセージを入力してください！").send()

@cl.on_message
async def on_message(message: cl.Message):
    print("入力されたメッセージ:" + message.content)
    await cl.Message(content="こんにちわ！").send()

In [ ]:
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.vectorstores import Chroma

#エンベッドの形式
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#チャットのモデル指定
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)
#プロンプトテンプレート
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#データベースにChroma
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)

#スタート時に表示させるメッセージ
@cl.on_chat_start
async def on_chat_start():
    await cl.Message(content="準備ができました！メッセージを入力してください！").send()

@cl.on_message
async def on_message(input_message: cl.Message):
    print("入力されたメッセージ:" + input_message.content)
    #データベースにキーワードで問い合わせ、結果はリスト
    documents = database.similarity_search(input_message.content)
    #ループ内で使う変数を初期化
    documents_string = ""
    #リストを走査
    for document in documents:
        documents_string += f"""
        ------------------------
        {document.page_content}
        """
    #問い合わせたドキュメント内で、キーワードで問い合わせ
    result = chat([
        HumanMessage(content=prompt.format(document=documents_string, query=input_message.content))
    ])
    #結果を返す
    await cl.Message(content=result.content).send()  

## チャット開始時にファイルをアップロードできるように

In [24]:
import os
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
chat = ChatOpenAI(
    model="gpt-3.5-turbo"
)
#
prompt= PromptTemplate(
    template="""文章を元に質問に答えてください。
    文章:
    {document}

    質問:{query}
    """,
    input_variables=["document", "query"]
)
#
text_splitter = SpacyTextSplitter(chunk_size=300, pipeline="ja_core_news_sm")



In [ ]:
#
import os
import chainlit as cl
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
from langchain.text_splitter import SpacyTextSplitter
from langchain.vectorstores import Chroma

#
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#
chat = ChatOpenAI(model="gpt-3.5-turbo")
#
prompt = PromptTemplate(template="""文章を元に質問に答えてください。 

文章: 
{document}

質問: {query}
""", input_variables=["document", "query"])

#
text_splitter = SpacyTextSplitter(chunk_size=300, pipeline="ja_core_news_sm")

@cl.on_chat_start
async def on_chat_start():
    #ファイルが選択されているか確認する変数
    files = None
    
    #ファイルが選択されるまで繰り返す
    while files is None:
        files = await cl.AskFileMessage(
            max_size_mb=20,
            content="PDFを選択してください",
            accept=["application/pdf"],
            raise_on_timeout=False,
        ).send()
    file = files[0]
    #tmpディレクトリが存在するか確認
    if not os.path.exists("tmp"):
        #なければ作成
        os.mkdir("tmp")
    #PDFファイルを保存する
    with open(f"tmp/{file.name}", "wb") as f:
        f.write(file.content)
        
    #保存したPDFファイルを読み込む
    documents = PyMuPDFLoader(f"tmp/{file.name}").load()
    #ドキュメントを分割する
    splitted_documents = text_splitter.split_documents(documents) 
     #データベースを初期化
    database = Chroma(
        embedding_function=embeddings,
        # 今回はpersist_directoryを指定しないことでデータベースの永続化を行わない
    )
    
    #ドキュメントをデータベースに追加する
    database.add_documents(splitted_documents) 

    cl.user_session.set(  #← データベースをセッションに保存する
        "database",  #← セッションに保存する名前
        database  #← セッションに保存する値
    )
    #
    await cl.Message(content=f"`{file.name}`の読み込みが完了しました。質問を入力してください。").send() #← 読み込み完了を通知する

@cl.on_message
async def on_message(input_message: cl.Message):
    print("入力されたメッセージ: " + input_message.content)
    #セッションからデータベースを取得する
    database = cl.user_session.get("database")
    #
    documents = database.similarity_search(input_message.content)

    documents_string = ""
    #
    for document in documents:
        documents_string += f"""
    ---------------------------
    {document.page_content}
    """

    result = chat([
        HumanMessage(content=prompt.format(document=documents_string, query=input_message.content))
    ])
    #
    await cl.Message(content=result.content).send()

## RetrievalQAを使う

In [22]:
#ChromaでＤＢに問い合わせＬＬＭで言語処理
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

#言語モデル指定
chat = ChatOpenAI(model="gpt-3.5-turbo")
#エンベッディングのモデル指定
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
#データベース指定
database = Chroma(
    persist_directory="./.data",
    embedding_function=embeddings
)
#データベースをRetrieverに
retriever = database.as_retriever()
#Retrieverに対し問い合わせてLLMで言語処理
qa = RetrievalQA.from_llm(
    llm=chat,
    retriever=retriever,
    return_source_documents=True
)

#質問
result = qa("飛行車の最高速度を教えて")
#結果表示
print(result["result"])
print("----------------------------------------")
#参照元のドキュメント
print(result["source_documents"])

都市部において飛行車が飛行する場合の最大速度は時速150キロメートルであり、都市部以外の地域において飛行車が飛行する場合の最大速度は時速250キロメートルです。
----------------------------------------
[Document(page_content='飛行車速度制限法\n第1条（目的）\n本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する\n基準を定めることを目的とする。\n\n\n\n\n第2条（定義）\n本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。\n\n\n\n\n第3条（一般的な速度制限）\n1.\n都市部において飛行車が飛行する場合の最大速度は、時速150キロメートルとする。\n\n\n2.\n都市部以外の地域において飛行車が飛行する場合の最大速度は、時速250キロメート\nルとする。\n\n\n3.\n特定の地域や施設の上空、または特定の飛行コース上では、別途速度制限が設けられ\nることがある。', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': './sample.pdf', 'format': 'PDF 1.4', 'keywords': '', 'modDate': "D:20230819044808Z00'00'", 'page': 3, 'producer': 'macOS バージョン13.5（ビルド22G74） Quartz PDFContext, AppendMode 1.1', 'source': './sample.pdf', 'subject': '', 'title': '飛行車に関する法制度', 'total_pages': 12, 'trapped': ''}), Document(page_content='飛行車速度制限法\n第1条（目的）\n本法は、飛行車の飛行安全及び一般公共の利益を確保するため、飛行車の飛行速度に関する\n基準を定めることを目的とする。\n\n\n\n\n第2条（定義）\n本法において「飛行車」とは、地上及び空中を移動する能力を有する車両を指す。\n\n\n\n\n第3条（一般的な速度制限）\n1.\n都市部

## Retrieverを使ってWikipediaを情報源に

In [20]:
from langchain.retrievers import WikipediaRetriever

#Wikipediaを日本語モードでRetrieverとして使う
retriever = WikipediaRetriever(
    lang="ja",
)
#Retrieverに問い合わせ
documents=retriever.get_relevant_documents(
    "大規模言語モデル"
)
#問い合わせた結果数
print(f"検索結果:{len(documents)}件")
#一件ずつ１００文字分表示
for document in documents:
    print("----------------取得したメタデータ----------------")
    print(document.metadata)
    print("----------------取得したテキスト----------------")
    print(document.page_content[:100])

C:\Users\hartm\anaconda3\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\hartm\anaconda3\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


検索結果:2件
----------------取得したメタデータ----------------
{'title': 'Microsoft Bing', 'summary': 'Microsoft Bing（マイクロソフト・ビング）は、Microsoftが提供するポータルサイトのひとつ。開発コンセプトに「意思決定を支援する検索エンジン」を掲げ、他の検索エンジンとの差別化を図っている。\n旧名称は Bing、MSN サーチ、Windows Live サーチ、Live サーチがあり、Windows Live サーチはWindows Liveサービスの一つでもあった。初期のコードネームはKumo（クモ）。\nBingの検索エンジンは、Googleとは異なる独自の検索技術を採用しているため、GoogleやYahoo! JAPANとは異なる検索結果を示す。プライバシー保護を謳う検索エンジンDuckDuckGoのソースとしても使用されている。\nBingの検索エンジンは、地方自治体の偽サイトが検索上位に表示されるなど、検索結果の精度や質については問題を指摘されている。\n2023年2月からは、検索エンジン「Bing」と生成AIチャットボット「Bing AI Chat」を組み合わせたサービスとして提供されている（後述。詳細は当該記事を参照）。', 'source': 'https://ja.wikipedia.org/wiki/Microsoft_Bing'}
----------------取得したテキスト----------------
Microsoft Bing（マイクロソフト・ビング）は、Microsoftが提供するポータルサイトのひとつ。開発コンセプトに「意思決定を支援する検索エンジン」を掲げ、他の検索エンジンとの差別化を図っ
----------------取得したメタデータ----------------
{'title': '言語モデル', 'summary': '言語モデル（げんごモデル、英: language model）は、単語列に対する確率分布を表わすものである。\n\n', 'source': 'https://ja.wikipedia.org/wiki/%E8%A8%80%E8%AA%9E%E3%83%A2%E3%83%87%E3%83%AB'}


In [18]:
#
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever

chat= ChatOpenAI()
#WikipediaRetriever日本語モード
retriever = WikipediaRetriever(
    lang="ja",
    doc_content_chars_max=500,
    top_k_results=2,
)
#Retrieverを使ってLLMで質問応答する
chain = RetrievalQA.from_llm(
    llm=chat,
    retriever=retriever,
    return_source_documents=True
)
#質問
result = chain("日本酒とは？")
#Wikipediaのソースからの結果リスト
source_documents=result["source_documents"]
#リスト件数表示
print(f"検索結果:{len(source_documents)}件")
#リストを一件づつ、１００文字分表示
for document in source_documents:
    print("----------------取得したメタデータ----------------")
    print(document.metadata)
    print("----------------取得したテキスト----------------")
    print(document.page_content[:100])
#全体の結果
print("----------------返答----------------")
print(result["result"])

C:\Users\hartm\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


検索結果:2件
----------------取得したメタデータ----------------
{'title': '日本酒', 'summary': '日本酒（にほんしゅ）、または和酒（わしゅ）は、通常は米（主に酒米）と麹と水を主な原料とする清酒（せいしゅ）を指す。日本特有の製法で醸造された酒で、醸造酒に分類される。\n\n', 'source': 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E9%85%92'}
----------------取得したテキスト----------------
日本酒（にほんしゅ）、または和酒（わしゅ）は、通常は米（主に酒米）と麹と水を主な原料とする清酒（せいしゅ）を指す。日本特有の製法で醸造された酒で、醸造酒に分類される。


== 定義 ==


===
----------------取得したメタデータ----------------
{'title': '日本酒の歴史', 'summary': '日本酒の歴史（にほんしゅのれきし）では、日本酒の歴史について説明する。', 'source': 'https://ja.wikipedia.org/wiki/%E6%97%A5%E6%9C%AC%E9%85%92%E3%81%AE%E6%AD%B4%E5%8F%B2'}
----------------取得したテキスト----------------
日本酒の歴史（にほんしゅのれきし）では、日本酒の歴史について説明する。


== 概要 ==

日本酒の製造には、酵母と米が酒へと発酵する過程が必須だが、酛は酒の母体となるもので「酒母」とも呼ばれる。
----------------返答----------------
日本酒（にほんしゅ）は、通常は米（主に酒米）と麹と水を主な原料とする清酒（せいしゅ）を指します。これは日本特有の製法で醸造された酒であり、醸造酒に分類されます。


In [19]:
#
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
from langchain.retrievers import RePhraseQueryRetriever
from langchain import LLMChain
from langchain.prompts import PromptTemplate

#
retriever = WikipediaRetriever(
    lang="ja",
    doc_content_chars_max=500,
)
#
llm_chain = LLMChain(
    llm=ChatOpenAI(
        temperature=0
    ),
    prompt=PromptTemplate(
        input_variables=["question"],
        template="""以下の質問からWikipediaで検索すべきキーワードを抽出してください。
        質問:{question}
        """
    )
)
#
re_phrase_query_retriever = RePhraseQueryRetriever(
    llm_chain=llm_chain,
    retriever=retriever,
)
#
documents = re_phrase_query_retriever.get_relevant_documents(
    #"私はだし巻き卵が好きです。ところで日本酒はありますか？"
    "私はラーメンが大好きです。ところでバーボンウイスキーとは何ですか？"
)

print(documents)

2024-05-10 23:32:24 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-05-10 23:32:24 - Re-phrased question: キーワード: バーボンウイスキー
[Document(page_content='スコッチ・ウイスキー（英語：Scotch whisky）は、イギリスのスコットランドで製造されるウイスキー。日本では世界5大ウイスキーの1つに数えられる。現在のイギリスでは後述の通り2009年スコッチ・ウイスキー規則により定義され、糖化から発酵、蒸留、熟成までスコットランドで行われたウィスキーのみがスコッチ・ウィスキーと呼ばれる。麦芽を乾燥させる際に燃焼させる泥炭（ピート）に由来する独特の煙のような香り（スモーキーフレーバーと呼ぶ）が特徴で、その香りの強さは銘柄によりまちまちである。ウイスキーはイギリスにとって主要な輸出品目の一つであり、その輸出規模は約200ヶ国、日本円にして6000億円（注: 以下で取り上げられる値に関して。2009年のポンド-円の為替相場は、1ポンド=約146円）。ウィスキーの全生産量のうち、スコッチ・ウイスキーは約7割を占める。\n\n\n== イギリスにおける法律上の定義 ==\n2009年スコッチ・ウイスキー規則（The Scotch Whisky Regulations 2009）により次のように定義されている。\nスコットランドにおいて製造されたウイスキーであって', metadata={'title': 'スコッチ・ウイスキー', 'summary': 'スコッチ・ウイスキー（英語：Scotch whisky）は、イギリスのスコットランドで製造されるウイスキー。日本では世界5大ウイスキーの1つに数えられる。現在のイギリスでは後述の通り2009年スコッチ・ウイスキー規則により定義され、糖化から発酵、蒸留、熟成までスコットランドで行われたウィスキーのみがスコッチ・ウィスキーと呼ばれる。麦芽を乾燥させる際に燃焼させる泥炭（ピート）に由来する独特の煙のような香り（スモーキーフレーバーと呼ぶ）が特徴で、その香りの強さは銘柄によりまちまちである。ウイスキーはイギリスにとって主要な